In [15]:
from dataset import TranslationDataset
from transformer import Transformer
import torch
from torch import nn
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torch.utils.data import DataLoader, Dataset
from torchtext.vocab import vocab
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import torch.optim as optim
import random
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [16]:
lang1 = 'en'
lang2 = 'de'
max_lines = None
num_test = 10
device = torch.device('mps')

In [17]:
lang1 = 'en'
lang2 = 'de'

en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')

def lang1_tokenizer(sentence):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [token.lower() for token in en_tokenizer(sentence)]

def lang2_tokenizer(sentence):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [token for token in de_tokenizer(sentence)]

def build_vocab(lang1='en', lang2='de'):
    counter1 = Counter()
    counter2 = Counter()
    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    for l in lines:
        l1, l2 = l.split('\t')
        counter1.update(lang1_tokenizer(l1))
        counter2.update(lang2_tokenizer(l2))

    vocab1 = vocab(counter1, min_freq=2, specials=['<unk>', '<pad>', '<sos>', '<eos>'])
    vocab2 = vocab(counter2, min_freq=2, specials=['<unk>', '<pad>', '<sos>', '<eos>'])
    vocab1.set_default_index(vocab1["<unk>"])
    vocab2.set_default_index(vocab2["<unk>"])
    return [vocab1, vocab2]

lang1_vocab, lang2_vocab = build_vocab(lang1, lang2)
lang1_vocab_size = len(lang1_vocab)
lang2_vocab_size = len(lang2_vocab)
print('lang1_vocab_vocab_size', lang1_vocab_size)
print('lang2_vocab_size', lang2_vocab_size)

lang1_vocab_vocab_size 12030
lang2_vocab_size 23470


In [18]:
dataset = TranslationDataset(lang1, lang2, max_lines, device, val=True)
test_loader = DataLoader(dataset, batch_size=1, shuffle=False)

d_model = 256  # Embedding Size
d_ff = 1024 # FeedForward dimension
d_k = d_v = 32  # dimension of K(=Q), V
n_layers = 3  # number of Encoder of Decoder Layer
n_heads = 4  # number of heads in Multi-Head Attention

model = Transformer(d_model, n_layers, len(lang1_vocab), len(lang2_vocab), d_k, d_v, n_heads, d_ff, device).to(device)
model.load_state_dict(torch.load('models/transformer.pt'))

for i in range(num_test):
    enc_inputs, target = next(iter(test_loader))
    enc_input = enc_inputs[0].view(1, -1)
    target = target[0].view(1, -1)
    predict = model.interface(enc_input, start_symbol=lang2_vocab['sos'], tgt_eos=lang2_vocab['eos'])
    print(' '.join(lang1_vocab.get_itos()[n.item()] for n in enc_input.squeeze()))
    print('> ' + ' '.join([lang2_vocab.get_itos()[n.item()] for n in target.squeeze()]))
    print('= ' + ' '.join([lang2_vocab.get_itos()[n.item()] for n in predict.squeeze()]))
    print('\n')

go . hi <unk> run ! wow
> Geh . <unk> Hallo <unk> !
= ! <unk>


go . hi <unk> run ! wow
> Geh . <unk> Hallo <unk> !
= ! die




RuntimeError: The size of tensor a (101) must match the size of tensor b (100) at non-singleton dimension 0